In [1]:
import numpy as np
import dask.array as da
import h5py
import bcolz
import sys
sys.path.insert(0, '../..')
%load_ext autoreload
%autoreload 1
%aimport allel.model.dask
%aimport allel.model.chunked
%aimport allel.model.ndarray
%aimport allel.model
%aimport allel
allel.__version__

'0.20.0.feature_dask'

In [2]:
callset = h5py.File('/data/coluzzi/ag1000g/data/phase1/release/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.h5', mode='r')
callset

<HDF5 file "ag1000g.phase1.ar3.pass.h5" (mode r)>

In [3]:
genotype = allel.model.chunked.GenotypeChunkedArray(callset['3L/calldata/genotype'])
genotype

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [4]:
# copy the first 2 million rows to use as benchmarking data
gc = genotype.copy(storage='hdf5mem_zlib1', stop=2000000, chunks=(5000, 100, 2))
gc

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [5]:
# how long does it take to read the data?
%time gc.max()

CPU times: user 6.63 s, sys: 8 ms, total: 6.64 s
Wall time: 6.63 s


3

In [6]:
%time gc.count_alleles(max_allele=3)

CPU times: user 9.04 s, sys: 8 ms, total: 9.05 s
Wall time: 8.96 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [7]:
gd = allel.model.dask.GenotypeDaskArray.from_array(gc)
gd

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [8]:
ac = gd.count_alleles(max_allele=3)
ac

,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [9]:
%time ac.compute(num_workers=1)

CPU times: user 10.9 s, sys: 572 ms, total: 11.5 s
Wall time: 11.2 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [10]:
%time ac.compute(num_workers=2)

CPU times: user 11.9 s, sys: 1.09 s, total: 13 s
Wall time: 7.41 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [11]:
%time ac.compute(num_workers=3)

CPU times: user 12 s, sys: 1.5 s, total: 13.5 s
Wall time: 7.16 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0
